In [3]:
import pandas as pd

In [4]:
df_roberta = pd.read_csv('dump_roberta_01.csv')
df_lexifield = pd.read_csv('dump_lexifield_01.csv')
df_heuristic = pd.read_csv('dump_heuristic_01.csv')

In [5]:
df_roberta.groupby('original_ref_type').mean()


/tmp/ipykernel_9348/1175137556.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_roberta.groupby('original_ref_type').mean()


,Unnamed: 0.1,level_0,Unnamed: 0,index,sentence_id,len,prec,rec
original_ref_type,,,,,,,,
Evoked\_Odorant,272.282051,283.846154,283.871795,642.435897,40.282051,49.692308,0.675094,0.439343
Odour\_Carrier,278.377049,387.032787,387.065574,875.983607,21.327869,44.688525,0.709310,0.656076
Quality,257.787330,271.570136,271.647059,616.266968,43.850679,46.058824,0.771731,0.641035
Smell\_Source,273.680233,315.459302,315.540698,718.267442,46.970930,34.697674,0.688879,0.709676
Smell\_Word,295.446809,302.063830,302.127660,686.106383,38.723404,29.765957,0.868351,0.957447


In [6]:
df_lexifield.groupby('original_ref_type').mean()

/tmp/ipykernel_9348/3776110344.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_lexifield.groupby('original_ref_type').mean()


,Unnamed: 0.1,level_0,Unnamed: 0,index,sentence_id,len,prec,rec,enriched_prec,enriched_rec
original_ref_type,,,,,,,,,,
Evoked\_Odorant,272.282051,283.846154,283.871795,642.435897,40.282051,49.692308,0.675094,0.439343,0.717025,0.494943
Odour\_Carrier,278.377049,387.032787,387.065574,875.983607,21.327869,44.688525,0.709310,0.656076,0.745125,0.722281
Quality,257.787330,271.570136,271.647059,616.266968,43.850679,46.058824,0.771731,0.641035,0.758033,0.709244
Smell\_Source,273.680233,315.459302,315.540698,718.267442,46.970930,34.697674,0.688879,0.709676,0.711483,0.763746
Smell\_Word,295.446809,302.063830,302.127660,686.106383,38.723404,29.765957,0.868351,0.957447,0.849253,0.978723


In [7]:
results = df_heuristic.groupby('original_ref_type').mean()

/tmp/ipykernel_9348/3358352588.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results = df_heuristic.groupby('original_ref_type').mean()


In [8]:
results['f1'] = results.apply(lambda x: 2*(x.enriched_prec * x.enriched_rec)/(x.enriched_prec + x.enriched_rec), axis=1)

In [9]:
results

,Unnamed: 0.1,level_0,Unnamed: 0,index,sentence_id,len,prec,rec,enriched_prec,enriched_rec,f1
original_ref_type,,,,,,,,,,,
Evoked\_Odorant,272.282051,283.846154,283.871795,642.435897,40.282051,49.692308,0.675094,0.439343,0.704204,0.500356,0.585031
Odour\_Carrier,278.377049,387.032787,387.065574,875.983607,21.327869,44.688525,0.709310,0.656076,0.746328,0.726522,0.736292
Quality,257.787330,271.570136,271.647059,616.266968,43.850679,46.058824,0.771731,0.641035,0.754106,0.714975,0.734019
Smell\_Source,273.680233,315.459302,315.540698,718.267442,46.970930,34.697674,0.688879,0.709676,0.711483,0.763746,0.736689
Smell\_Word,295.446809,302.063830,302.127660,686.106383,38.723404,29.765957,0.868351,0.957447,0.849253,0.978723,0.909403


In [10]:
import spacy
nlp = spacy.load("en_core_web_md")

/home/cboscher/.local/lib/python3.10/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/home/cboscher/.local/lib/python3.10/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [11]:
def pos_text(text, nlp):
    pos_text = nlp(text)
    
    text_with_pos = []
    for token in pos_text:
        text_with_pos.append([token.text, token.pos_])
    
    return text_with_pos

In [12]:
df_roberta['pos_pred'] = df_roberta['pred_token'].apply(lambda x: pos_text(x, nlp))
df_roberta['pos_true'] = df_roberta['clean_token'].apply(lambda x: pos_text(str(x), nlp))

df_lexifield['pos_pred'] = df_lexifield['pred_token'].apply(lambda x: pos_text(x, nlp))
df_lexifield['pos_true'] = df_lexifield['clean_token'].apply(lambda x: pos_text(str(x), nlp))

df_heuristic['pos_pred'] = df_heuristic['pred_token'].apply(lambda x: pos_text(x, nlp))
df_heuristic['pos_true'] = df_heuristic['clean_token'].apply(lambda x: pos_text(str(x), nlp))

In [13]:
df_roberta['pos_true']

0      [[stinking, VERB], [hot, ADJ], [stinking, VERB...
1      [[pulmonary, ADJ], [faces, VERB], [ozaena, PRO...
2                        [[poison, NOUN], [odour, NOUN]]
3      [[much, ADJ], [dung, NOUN], [sinne, NOUN], [st...
4      [[mere, ADJ], [plain, ADJ], [several, ADJ], [t...
                             ...                        
535    [[fumous, ADJ], [steamy, ADJ], [parboiled, VER...
536    [[aromatic, ADJ], [rather, ADV], [disagreeable...
537    [[musky, ADJ], [sexual, ADJ], [sphinx, NOUN], ...
538    [[rose, VERB], [lily, ADJ], [smell, NOUN], [sm...
539                     [[fragrant, ADJ], [smell, NOUN]]
Name: pos_true, Length: 540, dtype: object

In [14]:
df_roberta['pos_pred']

0      [[answer, VERB], [sweet, ADJ], [things, NOUN],...
1                      [[pulmonary, ADJ], [fetor, NOUN]]
2                                        [[odour, NOUN]]
3      [[yet, CCONJ], [no, DET], [dung, NOUN], [stink...
4      [[unpleasant, INTJ], [no, INTJ], [no, ADV], [l...
                             ...                        
535                      [[steamy, ADJ], [stench, NOUN]]
536    [[aromatic, ADJ], [rather, ADV], [disagreeable...
537    [[sphinx, NOUN], [moth, NOUN], [musky, NOUN], ...
538    [[rose, VERB], [lily, ADV], [vulgar, ADJ], [ca...
539                     [[fragrant, ADJ], [smell, NOUN]]
Name: pos_pred, Length: 540, dtype: object

In [15]:
set([x[1] for x in df_roberta['pos_pred'][0]]).union({'ADJ'})

{'ADJ', 'ADV', 'AUX', 'NOUN', 'PART', 'VERB'}

In [16]:
def compute_prec_by_pos(pos_pred, pos_true):
    pos_types = list(set(x[1] for x in pos_pred))
    
    scores = {}
    for pos_type in pos_types:
        len_pred = len([x for x in pos_pred if x[1]==pos_type])
        len_correct = len([x for x in pos_true if (x[1]==pos_type and x in pos_pred)])
        
                
        if len_pred == 0:
            continue
            
        scores[pos_type] = len_correct/len_pred
        
    return scores

In [33]:
def compute_rec_by_pos(pos_pred, pos_true):
    pos_types = list(set(x[1] for x in pos_pred))
    
    scores = {}
    for pos_type in pos_types:
        len_true = len([x for x in pos_true if x[1]==pos_type])
        len_correct = len([x for x in pos_true if (x[1]==pos_type and x in pos_pred)])
        
        if len_true == 0:
            continue
        scores[pos_type] = len_correct/len_true
        
    return scores

In [34]:
df_heuristic['prec_by_pos'] = df_heuristic.apply(lambda x : compute_prec_by_pos(x.pos_pred, x.pos_true), axis=1)
df_heuristic['rec_by_pos'] = df_heuristic.apply(lambda x : compute_rec_by_pos(x.pos_pred, x.pos_true), axis=1)

In [35]:
df = pd.DataFrame(df_heuristic['prec_by_pos'].to_list())
answer = dict(df.mean())

In [37]:
df = pd.DataFrame(df_heuristic['rec_by_pos'].to_list())
answer = dict(df.mean())

In [ ]:
a

In [28]:
df_lexifield['prec_by_pos'] = df_lexifield.apply(lambda x : compute_prec_by_pos(x.pos_pred, x.pos_true), axis=1)
df_lexifield['rec_by_pos'] = df_lexifield.apply(lambda x : compute_rec_by_pos(x.pos_pred, x.pos_true), axis=1)
df = pd.DataFrame(df_lexifield['prec_by_pos'].to_list())
answer = dict(df.mean())

In [31]:
answer

{'NOUN': 0.7462419599196901,
 'ADJ': 0.7382428940568475,
 'VERB': 0.7028423772609819,
 'NUM': 0.8333333333333333,
 'ADV': 0.78125,
 'PROPN': 0.7422201336675021,
 'DET': 1.0,
 'ADP': 1.0,
 'X': 1.0}

In [30]:
df = pd.DataFrame(df_lexifield['rec_by_pos'].to_list())
answer = dict(df.mean())

In [101]:
df = pd.DataFrame(df_roberta['rec_by_pos'].to_list())
answer = dict(df.mean())

In [102]:
answer

{'NOUN': 0.7062864256736274,
 'VERB': 0.6523611111111111,
 'ADJ': 0.8188693098384728,
 'NUM': 0.8333333333333333,
 'ADV': 0.8055555555555556,
 'PROPN': 0.7249707259953161,
 'DET': 1.0,
 'ADP': 1.0,
 'PART': 1.0,
 'X': 1.0}

In [21]:
dict(pd.DataFrame(df_roberta['pos_true'].apply(count_utterance).to_list()).sum())

{'VERB': 261.0,
 'ADJ': 549.0,
 'NOUN': 1112.0,
 'PROPN': 258.0,
 'NUM': 12.0,
 'ADV': 36.0,
 'DET': 5.0,
 'SCONJ': 5.0,
 'ADP': 13.0,
 'PART': 3.0,
 'INTJ': 1.0,
 'X': 3.0,
 'PUNCT': 3.0,
 'PRON': 1.0}

In [18]:
def count_utterance(tab):
    dict = {}
    
    for x in tab:
        if not x[1] in dict.keys():
            dict[x[1]]=1
        else:
            dict[x[1]]+=1
    return dict

In [ ]:
df_robe